# Creating Agents with Persistence & Streaming using LangGraph by `Mr. Harshit Dawar`
* **Persistence** helps save the state of the Agent at a given point in time, this helps a lot in long running application to get back to a previous state of the Agent & continue from there!
* **Streaming** in LangGraph can either stream the complete events or the tokens as well!

In [1]:
# Importing the required Libraries
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_tavily import TavilySearch
from dotenv import load_dotenv
# Importing the SQLite Saver to save the state, in this practical, state of the Agent is stored in Memory
from langgraph.checkpoint.sqlite import SqliteSaver

# Importing the OpenAI & Tavily Keys required for the practical
load_dotenv()

True

In [5]:
# Initializing the Tavily Search Tool
tool = TavilySearch(max_results=5)
print(type(tool))
print(tool.name)

<class 'langchain_tavily.tavily_search.TavilySearch'>
tavily_search


In [6]:
# Creating a class to store the AgentState
# "operator.add" is used to append the ongoing the messages in the Agent state, so that the Agent can remember what discussions has been done

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [7]:
# Definning the Agent class
class Agent:
    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer)    # Adding the checkpoint here
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [9]:
prompt = """You are a smart research assistant. Use the search engine to look up information.
You are allowed to make multiple calls (either together or in sequence).
Only look up information when you are sure of what you want.
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(model="gpt-5-nano")

## Streaming Events with Persistence
* Persistence is controlled by the Checkpoint & Thread

In [14]:
with SqliteSaver.from_conn_string(":memory:") as memory:
    MyAgent = Agent(model, [tool], system=prompt, checkpointer=memory)
    messages = [HumanMessage(content="What is the weather in new delhi?")]
    thread = {"configurable": {"thread_id": "1"}}
    for event in MyAgent.graph.stream({"messages": messages}, thread):
        for v in event.values():
            print(v['messages'])

    messages = [HumanMessage(content="What is the population there?")]
    for event in MyAgent.graph.stream({"messages": messages}, thread):
        for v in event.values():
            print(v['messages'])

[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_V8qlAA3XXaIrC6J84VivTqWV', 'function': {'arguments': '{"query":"New Delhi current weather today","time_range":"day","search_depth":"advanced"}', 'name': 'tavily_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 487, 'prompt_tokens': 1426, 'total_tokens': 1913, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 448, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CV2Bw9YQgNWrA1keHDiUQeUgmi4ua', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--3d4f6df4-ab84-4fb3-a911-32763455841b-0', tool_calls=[{'name': 'tavily_search', 'args': {'query': 'New Delhi current weather today', 'time_range': 'day', 'search_depth': 'advanced'}, 'id': 'call_V8qlAA3XXa

## Streaming Tokens

In [18]:
from langgraph.checkpoint.sqlite.aio import AsyncSqliteSaver

async with AsyncSqliteSaver.from_conn_string(":memory:") as memory:
    MyAgent = Agent(model, [tool], system=prompt, checkpointer=memory)
    messages = [HumanMessage(content="What is the weather in New Delhi?, tell me the live current temperature and population as well")]
    thread = {"configurable": {"thread_id": "5"}}
    async for event in MyAgent.graph.astream_events({"messages": messages}, thread, version="v1"):
        kind = event["event"]
        if kind == "on_chat_model_stream":
            content = event["data"]["chunk"].content
            if content:
                # Empty content in the context of OpenAI means
                # that the model is asking for a tool to be invoked.
                # So we only print non-empty content
                print(content, end=" $ ")   # Using "$" for seperation between the tokens

Calling: {'name': 'tavily_search', 'args': {'query': 'New Delhi weather current temperature live', 'search_depth': 'advanced', 'include_images': False, 'start_date': None, 'end_date': None, 'time_range': None, 'include_favicon': False, 'exclude_domains': None, 'include_domains': None, 'topic': 'general'}, 'id': 'call_OZvTTzn8e0GAYTBJJMBmU5BQ', 'type': 'tool_call'}
Calling: {'name': 'tavily_search', 'args': {'query': 'New Delhi population city proper 2023 estimate', 'search_depth': 'advanced', 'include_images': False, 'start_date': None, 'end_date': None, 'time_range': None, 'include_favicon': False, 'exclude_domains': None, 'include_domains': None, 'topic': 'general'}, 'id': 'call_RiCWj4UvsyZyLC9RGGCFgaDm', 'type': 'tool_call'}
Back to the model!
Calling: {'name': 'tavily_search', 'args': {'query': 'New Delhi current temperature now live', 'search_depth': 'advanced', 'include_images': False, 'start_date': None, 'end_date': None, 'time_range': None, 'include_favicon': False, 'exclude_do